In [2]:
import pandas as pd
import json
import requests 
import time
import random
import tqdm
from urllib.parse import urlparse

In [3]:
# cookies = {
#     '_trackity' : '74634137-8702-8b12-fe42-46a46a2a2573',
#     'TOKENS' : '{%22access_token%22:%22Ae1PtIuUDbozkgpQ6VOW57jKqvchZxF2%22%2C%22expires_in%22:157680000%2C%22expires_at%22:1844582643238%2C%22guest_token%22:%22Ae1PtIuUDbozkgpQ6VOW57jKqvchZxF2%22}',
#     'delivery_zone' : 'Vk4wMzkwMDYwMDE=',
#     '_gid' : 'GA1.2.1038835342.1686902648',
#     'tiki_client_id' : '1000198359.1686902648',
#     '_gcl_au' : '1.1.712561686.1686902651',
#     '_fbp' : 'fb.1.1686902651246.1147493922',
#     '_hjFirstSeen' : '1', 
#     '_hjIncludedInSessionSample_522327' : '0', 
#     '_hjSession_522327':'eyJpZCI6IjRmYmI5NDEzLTc1MzAtNDA3Yy1iYTMyLTYzZTI4Y2MzZTA5NSIsImNyZWF0ZWQiOjE2ODY5MDI2NTEzMTksImluU2FtcGxlIjpmYWxzZX0=',
#     '_hjAbsoluteSessionInProgress' : '0',
#     '__uidac' : 'b0f5fe8e77ade31f60d53549702ce9ac',
#     '__iid' : '749', '__iid' : '749', '__su' : '0',
#     '__su':'0', '__RC' : '5', '__R' : '3',
#     '_hjSessionUser_522327':'eyJpZCI6IjFiMTI5YzM0LWE2NTItNWZmOC05NmEzLTFhY2JlOTMwZTViYyIsImNyZWF0ZWQiOjE2ODY5MDI2NTEzMDIsImV4aXN0aW5nIjp0cnVlfQ==',
#     '__tb':'0', 
#     '__IP' : '712479999',
#     'TIKI_RECOMMENDATION' : '40be1840f8c66757ac3812da38a02d7b',
#     'TKSESSID':'cf67caae16e0362971a2fe1e3367c12e',
#     '_bs':'c4f97c3c-1a3b-0d7a-f2dd-a0a3516f6617',
#     '_gat' : '1',
#     '_ga': 'GA1.2.1000198359.1686902648',
#     '__uif' :'__uid%3A3569195332985378685%7C__ui%3A1%252C5%7C__create%3A1669195332',
#     'cto_bundle':'5mSn7195TEwza3VCTEZZTk9HdWw1cSUyQkM2dGRGV0ZtUlRua1NvRjV5S2phc096S1lIVlJySjlmZkdqUVNScWNMbEdoamc0bE82czlSR0lYS1pKMU1pNk40U2lnQSUyRiUyQlBGbFJ5MXBwUUtwOW1oVUoxQm9hY1olMkZ1ZWplRzhUdThiZnNqeExlMnkyV1NyMnpBNmlJdXAwRFZvb2FsQSUzRCUzRA',
#     'amp_99d374':'Rz66229-5rdmltAOjfyUvJ...1h31jn9b2.1h31k9l0t.2j.5e.81',
#     '_ga_GSD4ETCY1D':'GS1.1.1686902651.1.1.1686903252.28.0.0'
# }


headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
    'Accept':'application/json, text/plain, */*',
    'Accept-Language': 'en-US,en;q=0.9,vi;q=0.8,th;q=0.7,zh-CN;q=0.6,zh;q=0.5',
    'X-Guest-Token': 'Ae1PtIuUDbozkgpQ6VOW57jKqvchZxF2',
    'Connection' : 'keep-alive',
    'TE': 'Trailers'
}

In [4]:
list_url = [
            'https://tiki.vn/giay-cao-got-nhon/c49542',
            'https://tiki.vn/giay-cao-got-ho-mui/c49548',
            'https://tiki.vn/giay-the-thao-co-thap/c49552',
            'https://tiki.vn/dep-quai-ngang/c49578'
            ]

page_error = []
price_range = [
    '500,15000', '15001,30000', '30001,50000', '50001,80000', '80001,150000',
    '150001,250000', '250001,350000', '350001,550000', '550000, 100000000'
                ]
list_page_50 = []

for url in list_url:
    parsed_url = urlparse(url)
    path_segments = parsed_url.path.split('/')

    category_name = path_segments[1]
    category_id = path_segments[2][1:]
    product_data = []
    for price in price_range:
        params = {
            'limit': '40',
            'include': 'advertisement',
            'aggregations': '2',
            'trackity_id': '74634137-8702-8b12-fe42-46a46a2a2573',
            'category': category_id,
            'page': '1',
            'urlKey': category_name,
            'price': price
        }
        
        response = requests.get('https://tiki.vn/api/personalish/v1/blocks/listings', headers=headers, params=params)

        if response.status_code == 200:
            response_json = response.json()
            last_page = response_json.get('paging', {}).get('last_page')
            print('CATEGORY', category_name, price)
            print('total pages', last_page)

            if last_page == 50:
                list_page_50.append([category_name, price])
                continue
            else:
                for i in range(1, last_page + 1):
                    params['page'] = i
                    response = requests.get('https://tiki.vn/api/personalish/v1/blocks/listings', headers=headers,params=params)

                    if response.status_code == 200:
                        print('request done page', i)
                        response_json = response.json()
                        for record in response_json.get('data', []):
                            product_id = record.get('id')
                            product_data.append({'product_id': product_id, 'category_name': category_name,
                                                 'max_page': last_page})
                    else:
                        print('cannot request page', i)
                        page_error.append([category_name,category_id,price,i])
                    time.sleep(random.randrange(3, 10))

    product_df = pd.DataFrame(product_data, columns=['product_id', 'category_name', 'max_page'])
    output_file_name = f'GIAY-NU-50-{category_name}.csv'
    product_df.to_csv(output_file_name, index=False)


CATEGORY giay-cao-got-nhon 500,15000
total pages 1
request done page 1
CATEGORY giay-cao-got-nhon 15001,30000
total pages 1
request done page 1
CATEGORY giay-cao-got-nhon 30001,50000
total pages 1
request done page 1
CATEGORY giay-cao-got-nhon 50001,80000
total pages 1
request done page 1
CATEGORY giay-cao-got-nhon 80001,150000
total pages 4
request done page 1
request done page 2
request done page 3
request done page 4
CATEGORY giay-cao-got-nhon 150001,250000
total pages 11
request done page 1
request done page 2
request done page 3
request done page 4
request done page 5
request done page 6
request done page 7
request done page 8
request done page 9
request done page 10
request done page 11
CATEGORY giay-cao-got-nhon 250001,350000
total pages 9
request done page 1
request done page 2
request done page 3
request done page 4
request done page 5
request done page 6
request done page 7
request done page 8
request done page 9
CATEGORY giay-cao-got-nhon 350001,550000
total pages 11
request

In [5]:
print(list_page_50)

[]


In [1]:
print(page_error)

NameError: name 'page_error' is not defined

In [ ]:
product_data_error = []
if len(page_error) != 0:
    for i in range(len(page_error)):
        error_page_params = {
                    'limit': '40',
                    'include': 'advertisement',
                    'aggregations': '2',
                    'trackity_id': '74634137-8702-8b12-fe42-46a46a2a2573',
                    'category': page_error[i][1],
                    'page': page_error[i][3],
                    'urlKey': page_error[i][0],
                    'price': page_error[i][2]
                }

        response = requests.get('https://tiki.vn/api/personalish/v1/blocks/listings', headers=headers,params=params)

        if response.status_code == 200:
            print('request done page', page_error[i][0], page_error[i][3], page_error[i][2])
            response_json = response.json()
            for record in response_json.get('data', []):
                product_id = record.get('id')
                product_data_error.append({'product_id': product_id, 'category_name': page_error[i][0]})

        time.sleep(random.randrange(3, 10))

    product_df = pd.DataFrame(product_data_error, columns=['product_id', 'category_name','max_range'])
    output_file_name = f'GIAY-NU-50-ERROR.csv'
    product_df.to_csv(output_file_name, index=False)
        